In [1]:
import tensorflow as tf
#tf.debugging.set_log_device_placement(True)
physical_devices = tf.config.list_physical_devices('GPU')
gpuSelected = 1

tf.config.set_visible_devices([], 'GPU')
tf.config.set_visible_devices(tf.config.list_physical_devices('GPU')[gpuSelected], 'GPU')
 
try: 
    tf.config.experimental.set_memory_growth(physical_devices[gpuSelected], True)
    
    print("memory growth ok")
    print("tensorflow version = %s" % tf.__version__)
    print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
    print("GPU %d selected" % gpuSelected)
except: 
    print("erro")
    pass

memory growth ok
tensorflow version = 2.1.0
Num GPUs Available:  4
GPU 1 selected


In [1]:
import tensorflow as tf
print(tf.keras.__version__)

2.2.4-tf


In [2]:
import scipy
from tensorflow.keras.layers import Input, Dense, Reshape, Flatten, Dropout, Concatenate
from tensorflow.keras.layers import BatchNormalization, Activation, ZeroPadding2D, Embedding
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import UpSampling2D, Conv2D
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
import datetime
import matplotlib.pyplot as plt
import sys
import numpy as np
import os
import scipy
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from tensorflow.keras.layers import concatenate
import shutil
sys.path.append("/data/acunha/LungDL/JDIArtigo/Eduardo/Brain/")


from DataLoader import DataLoader
print("imports ok")

imports ok
imports ok


In [3]:
class MMIGAN():
    def __init__(self, img_shape, pathOfResult, dataLoader):        # Input shape
         
        self.img_shape = img_shape
        self.pathOfResult = pathOfResult
        self.num_classes = 3
        
        self.generatorInteration = 3
        self.loss_weights = [1, 1, 100]
        self.data_loader =  dataLoader
        
        self.pathOfResult = pathOfResult


        # Calculate output shape of D (PatchGAN)
        patch = int(self.img_shape[0] / 2**4)
        self.disc_patch = (patch, patch, 1)

        # Number of filters in the first layer of G and D
        self.gf = 64
        self.df = 64
         
        losses = ['mse','sparse_categorical_crossentropy']
        optimizer = Adam( 0.0002, 0.5, 0.999)
        #optimizer = RMSprop(lr=0.001, rho=0.9, epsilon=None, decay=0.0)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        #self.discriminator.summary()
        self.discriminator.compile(loss=losses,
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()
        #self.generator.summary()
        
        # Input images and their conditioning images
        img_A = Input(shape=self.img_shape)
        img_B = Input(shape=self.img_shape)
        label = Input(shape=(1,))
        # By conditioning on B generate a fake version of A
        
        fake_A = self.generator([img_B, label])

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # Discriminators determines validity of translated images / condition pairs
        valid, target_label = self.discriminator([fake_A, img_B])

        self.combined = Model(inputs=[img_A, img_B, label], outputs=[valid, target_label, fake_A])
        self.combined.compile(loss=['mse', 'sparse_categorical_crossentropy', 'mae'],
                              loss_weights=self.loss_weights,
                              optimizer=optimizer)
     
    def build_generator(self):
        """U-Net Generator"""

        def conv2d(layer_input, filters, f_size=4, bn=True):
            """Layers used during downsampling"""
            d = Conv2D(filters, kernel_size=f_size, strides=2, padding='same')(layer_input)
            d = LeakyReLU(alpha=0.2)(d)
            if bn:
                d = BatchNormalization(momentum=0.8)(d)
            return d

        def deconv2d(layer_input, skip_input, filters, f_size=4, dropout_rate=0):
            """Layers used during upsampling"""
            u = UpSampling2D(size=2)(layer_input)
            u = Conv2D(filters, kernel_size=f_size, strides=1, padding='same', activation='relu')(u)
            if dropout_rate:
                u = Dropout(dropout_rate)(u)
            u = BatchNormalization(momentum=0.8)(u)
            u = Concatenate()([u, skip_input])
            return u
       
        inputA = Input(shape=self.img_shape)
        inputB = Input(shape=(1,))

        d1 = conv2d(inputA, self.gf, bn=False)
        d2 = conv2d(d1, self.gf*2)
        d3 = conv2d(d2, self.gf*4)
        d4 = conv2d(d3, self.gf*8)
        x = Model(inputA, d4)
        print("gerador input A")
        x.summary()
        
        
        
        y = Flatten()(Embedding(self.num_classes, 8*8*self.gf*4)(inputB))
        y = Dense(8*8*self.gf*4)(y)
        y = BatchNormalization()(y)
        y = Reshape((8, 8, self.gf*4))(y)
        y = BatchNormalization()(y)
        y = LeakyReLU()(y)
        y = UpSampling2D()(y)
        y = Conv2D(self.gf*8, kernel_size=5, padding="same")(y)
        y = BatchNormalization()(y)
        y = LeakyReLU()(y)

        y = Model(inputs=inputB, outputs=y)
        print("gerador input B")
        y.summary()
        
        
        print("combinado A + B")
        
        combined = concatenate([x.output, y.output])
        d5 = conv2d(combined, self.gf*16)
        u1 = deconv2d(d5, d4, self.gf*8)
        u2 = deconv2d(u1, d3, self.gf*4)
        u3 = deconv2d(u2, d2, self.gf*2)
        u4 = deconv2d(u3, d1, self.gf*1)
        u5 = UpSampling2D(size=2)(u4) 
        output1 = Conv2D(1, kernel_size=4, strides=1, padding='same', activation='tanh')(u5)

        ###
        
        """
        d21 = conv2d(output1, self.gf, bn=False)
        d22 = conv2d(d21, self.gf*2)
        d23 = conv2d(d22, self.gf*4)
        d24 = conv2d(d23, self.gf*8)
        
        u21 = deconv2d(d24, d23, self.gf*4)
        u22 = deconv2d(u21, d22, self.gf*2)
        u23 = deconv2d(u22, d21, self.gf*1)
        u24 = UpSampling2D(size=2)(u23) 
        output_img = Conv2D(1, kernel_size=4, strides=1, padding='same', activation='tanh')(u24)
        
        model = Model([inputA, inputB], output_img)
        """
        ###
        model = Model([inputA, inputB], output1)
         
        print("Generator")
        print()
        print()
        model.summary()
        return model

    def build_discriminator(self):

        def d_layer(layer_input, filters, f_size=4, bn=True):
            """Discriminator layer"""
            d = Conv2D(filters, kernel_size=f_size, strides=2, padding='same')(layer_input)
            d = LeakyReLU(alpha=0.2)(d)
            if bn:
                d = BatchNormalization(momentum=0.8)(d)
            return d

        
        img_A = Input(shape=self.img_shape)
        img_B = Input(shape=self.img_shape)

        # Concatenate image and conditioning image by channels to produce input
        combined_imgs = Concatenate(axis=-1)([img_A, img_B])

        d1 = d_layer(combined_imgs, self.df, bn=False)
        d2 = d_layer(d1, self.df*2)
        d3 = d_layer(d2, self.df*4)
        d4 = d_layer(d3, self.df*8)

        validity = Conv2D(1, kernel_size=4, strides=1, padding='same')(d4)
        
        d5 = Flatten()(d4)
        label = Dense(self.num_classes, activation="softmax")(d5)
        
        m = Model([img_A, img_B], d5)
        
        model = Model([img_A, img_B], [validity, label])
        
        print("discriminator")
        
        
        return model
    
    

    def train(self, epochs, batch_size=1, sample_interval=500):
        
        os.makedirs(self.pathOfResult + "/logs", exist_ok=True)
        start_time = datetime.datetime.now()

        # Adversarial loss ground truths
        valid = np.ones((batch_size,) + self.disc_patch)
        fake = np.zeros((batch_size,) + self.disc_patch)
        #target_label = np.ones((batch_size,1))
        

        for epoch in range(epochs):

            dlossPlot = []
            glossPlot = []
            accPlot = []
            
            for batch_i, (imgs_A, imgs_B, imgs_C) in enumerate(self.data_loader.loadBatchPaired(batch_size)):
               
                targetLabels = np.random.randint(0, 3, (batch_size, 1))
                inputLabels = np.random.randint(0, 3, (batch_size, 1))
                for j in range(0, len(targetLabels)):
                    while(targetLabels[j] == inputLabels[j]):
                      aux = np.random.randint(0, 3, (1, 1))
                      targetLabels[j] = aux[0]
                
                inputImage = []
                targetImage = []
                
                for i in range(0 , batch_size):
                    
                    if inputLabels[i] == 0:
                      #print("InputLabel = 0 -> InputImage = T1")
                      inputImage.append(imgs_A[i])
                      
                    elif inputLabels[i] == 1:
                      #print("InputLabel = 1 -> InputImage = T2")
                      inputImage.append(imgs_B[i])
                      
                    else:
                      #print("InputLabel = 2 -> InputImage = PD")
                      inputImage.append(imgs_C[i])
                      
                    if targetLabels[i] == 0:
                      #print("TargetLabel = 0 -> TargetImage = PD")
                      targetImage.append(imgs_A[i])
                    
                    elif targetLabels[i] == 1:
                      #print("TargetLabel = 1 -> TargetImage = T2")
                      targetImage.append(imgs_B[i])
                    
                    else:
                      #print("TargetLabel = 2 -> TargetImage = PD")
                      targetImage.append(imgs_C[i])
                        
               
                targetImage = np.asarray(targetImage)
                inputImage = np.asarray(inputImage)
                
                
                fakeImages = self.generator.predict([inputImage, targetLabels])

                d_loss_real = self.discriminator.train_on_batch([targetImage, inputImage], [valid, targetLabels] )
                d_loss_fake = self.discriminator.train_on_batch([fakeImages, inputImage], [fake, targetLabels] )
                
                d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
                
                
                
                
                # -----------------
                #  Train Generator
                # -----------------

                # Train the generators
                for interaction in range(self.generatorInteration):
                  g_loss = self.combined.train_on_batch([targetImage, inputImage, targetLabels], [valid, targetLabels, targetImage])
                
                dlossPlot = np.append(dlossPlot, d_loss[0])
                glossPlot = np.append(glossPlot, g_loss[0])
                accPlot = np.append(accPlot, 100*d_loss[1])
                

                elapsed_time = datetime.datetime.now() - start_time
               
                
                # Plot the progress
                f = open(self.pathOfResult + "/logs/%d.txt" % epoch, "a+")
                f.write("[Epoch %d/%d] [Batch %d/%d] [D loss: %f, acc: %3d%%, op_acc: %.2f%%] [G loss: %f] time: %s" % (epoch, epochs,
                                                                        batch_i, self.data_loader.n_batches,
                                                                        d_loss[0], 100*d_loss[3], 100*d_loss[4] ,
                                                                        g_loss[0],
                                                                        elapsed_time))

                #f.close() 
                print ("[Epoch %d/%d] [Batch %d/%d] [D loss: %f, acc: %3d%%, op_acc: %.2f%%] [G loss: %f] time: %s" % (epoch, epochs,
                                                                        batch_i, self.data_loader.n_batches,
                                                                        d_loss[0], 100*d_loss[3], 100*d_loss[4] ,
                                                                        g_loss[0],
                                                                        elapsed_time))
                
                # If at save interval => save generated image samples
                if batch_i % sample_interval == 0:
                    self.sample_images(epoch, batch_i, self.pathOfResult)
                    
                if batch_i == int(self.data_loader.n_batches - 10) or batch_i == 2:
                    os.makedirs(pathOfResult, exist_ok=True)
                    print("save generator")
                    os.makedirs(pathOfResult + '/generator' , exist_ok=True)
                    self.generator.save(pathOfResult + '/generator/%d.h5' % epoch) 
   
    def sample_images(self, epoch, batch_i, pathOfResult):
        os.makedirs(pathOfResult, exist_ok=True)
        
        #save generator 
        
        
        imgs_A, imgs_B, imgs_C = self.data_loader.loadBatchTest(batch_size=1)


        targetLabels = np.random.randint(0, 3, (1, 1))
        inputLabels = np.random.randint(0, 3, (1, 1))
        for j in range(0, len(targetLabels)):
            while(targetLabels[j] == inputLabels[j]):
              aux = np.random.randint(0, 3, (1, 1))
              targetLabels[j] = aux[0]

        inputImage = []
        targetImage = []

        for i in range(0 , 1):

            if inputLabels[i] == 0:
                inputImage.append(imgs_A[i])

            elif inputLabels[i] == 1:
                inputImage.append(imgs_B[i])

            else:
                inputImage.append(imgs_C[i])

            if targetLabels[i] == 0:
                targetImage.append(imgs_A[i])

            elif targetLabels[i] == 1:
                targetImage.append(imgs_B[i])

            else:
                targetImage.append(imgs_C[i])


        #print(inputLabels)
        #print(targetLabels)
        #print()

        targetImage = np.asarray(targetImage)
        inputImage = np.asarray(inputImage)



        fake = self.generator.predict([inputImage, targetLabels])

        #gen_imgs = np.concatenate([imgs_B, fake_A, imgs_A])

        aux = np.concatenate((targetImage[0][:, :, 0], inputImage[0][:, :, 0]), axis=1)
        aux = np.concatenate((aux, fake[0][:, :, 0]), axis=1)
        #aux = Image.fromarray(aux)
        #aux.save("%s/%d_%d-1.png" % (pathOfResult, epoch, batch_i))
        print("SAVE")
        if inputLabels[0] == 0 and targetLabels[0] == 1:
          plt.imsave("%s/%d_%d-T1-T2.png" % (pathOfResult, epoch, batch_i), aux, cmap="gray")
          print("%d_%d-T1-T2.png" % ( epoch, batch_i))

        elif inputLabels[0] == 0 and targetLabels[0] == 2:
          plt.imsave("%s/%d_%d-T1-PD.png" % (pathOfResult, epoch, batch_i), aux, cmap="gray")        
          print("%d_%d-T1-PD.png" % ( epoch, batch_i))

        elif inputLabels[0] == 1 and targetLabels[0] == 0:
          plt.imsave("%s/%d_%d-T2-T1.png" % (pathOfResult, epoch, batch_i), aux, cmap="gray")
          print("%d_%d-T2-T1.png" % ( epoch, batch_i))
        elif inputLabels[0] == 1 and targetLabels[0] == 2:
          plt.imsave("%s/%d_%d-T2-PD.png" % (pathOfResult, epoch, batch_i), aux, cmap="gray")
          print("%d_%d-T2-PD.png" % ( epoch, batch_i))
        elif inputLabels[0] == 2 and targetLabels[0] == 0:
          plt.imsave("%s/%d_%d-PD-T1.png" % (pathOfResult, epoch, batch_i), aux, cmap="gray")
          print("%d_%d-PD-T1.png" % (epoch, batch_i))
        else:
          plt.imsave("%s/%d_%d-PD-T2.png" % (pathOfResult, epoch, batch_i), aux, cmap="gray")
          print("%d_%d-PD-T2.png" % ( epoch, batch_i))
              
        plt.close()




In [ ]:
if __name__ == '__main__':
    typeA = "T1"
    typeB = "T2"
    typeC = "PD"

    pathImageTrainAPaired = "/data/acunha/LungDL/MMIGAN/datasets/IOP/%s" % (typeA)
    pathImageTrainAUnpaired = "/data/acunha/LungDL/MMIGAN/datasets/IOP/%s" % (typeA)
    pathOfResult= "/data/acunha/LungDL/MMIGAN/Results-MMIGAN-IOP-JOURNAL-%s-%s-%s" % (typeA, typeB, typeC)

    imageRegion=""
    
    epochs = 30
    batch_size = 10
    sample_interval = 200
    img_shape = (256, 256, 1)
    
    dataLoader = DataLoader(img_res= img_shape, pathImageTrainAPaired= pathImageTrainAPaired, pathImageTrainAUnpaired=pathImageTrainAUnpaired, typeA=typeA,
              typeB=typeB, typeC=typeC, imageRegion=imageRegion, seed=0)
    dataLoader.holdOut(60)
    
    gan = MMIGAN(img_shape=img_shape, pathOfResult= pathOfResult, dataLoader= dataLoader)
    gan.train(epochs=epochs, batch_size=batch_size, sample_interval=sample_interval)

Pareadas| Temos um total de 8160 imagens de T1, 8160 de T2 e 8160 de PD para treinamento, os pacientes ['035', '230', '231', '232', '233', '234', '238', '290', '291', '292', '293', '294', '303', '305', '306', '307', '310', '314', '315', '322', '331', '332', '333', '337', '340', '345', '347', '371', '372', '373', '378', '382', '388', '395', '423', '424', '425', '426', '427', '430', '433', '434', '435', '442', '462', '463', '464', '469', '470', '473', '474', '475', '476', '477', '478', '510', '517', '532', '541', '542'] foram selecionados

Pareadas| Temos um total de 1768 imagens de T1, 1768 de T2 e 1768 de PD para teste, os pacients ['543', '547', '548', '553', '561', '563', '571', '573', '574', '588', '595', '596', '597'] foram selecionados

Nao Pareadas| Temos um total de 8160 imagens de T1, 8160 de T2 e 8160 de PD para treinamento, os pacientes ['035', '230', '231', '232', '233', '234', '238', '290', '291', '292', '293', '294', '303', '305', '306', '307', '310', '314', '315', '322', 

conv2d_12 (Conv2D)              (None, 32, 32, 256)  4194560     up_sampling2d_2[0][0]            
__________________________________________________________________________________________________
batch_normalization_11 (BatchNo (None, 32, 32, 256)  1024        conv2d_12[0][0]                  
__________________________________________________________________________________________________
concatenate_3 (Concatenate)     (None, 32, 32, 512)  0           batch_normalization_11[0][0]     
                                                                 batch_normalization_4[0][0]      
__________________________________________________________________________________________________
up_sampling2d_3 (UpSampling2D)  (None, 64, 64, 512)  0           concatenate_3[0][0]              
__________________________________________________________________________________________________
conv2d_13 (Conv2D)              (None, 64, 64, 128)  1048704     up_sampling2d_3[0][0]            
__________

[Epoch 0/30] [Batch 45/816] [D loss: 4.790597, acc:  50%, op_acc: 70.00%] [G loss: 18.029097] time: 0:01:54.635941
[Epoch 0/30] [Batch 46/816] [D loss: 4.044944, acc:  54%, op_acc: 90.00%] [G loss: 17.379967] time: 0:01:56.915336
[Epoch 0/30] [Batch 47/816] [D loss: 4.920577, acc:  51%, op_acc: 80.00%] [G loss: 22.056871] time: 0:01:59.157380
[Epoch 0/30] [Batch 48/816] [D loss: 5.053272, acc:  53%, op_acc: 65.00%] [G loss: 15.160204] time: 0:02:01.246192
[Epoch 0/30] [Batch 49/816] [D loss: 4.636422, acc:  52%, op_acc: 80.00%] [G loss: 24.449944] time: 0:02:03.475251
[Epoch 0/30] [Batch 50/816] [D loss: 4.301491, acc:  49%, op_acc: 80.00%] [G loss: 21.533125] time: 0:02:05.661006
[Epoch 0/30] [Batch 51/816] [D loss: 1.030260, acc:  53%, op_acc: 95.00%] [G loss: 19.462782] time: 0:02:07.866574
[Epoch 0/30] [Batch 52/816] [D loss: 1.135785, acc:  60%, op_acc: 95.00%] [G loss: 17.589943] time: 0:02:10.063122
[Epoch 0/30] [Batch 53/816] [D loss: 1.572554, acc:  52%, op_acc: 90.00%] [G los

[Epoch 0/30] [Batch 117/816] [D loss: 0.392846, acc:  49%, op_acc: 95.00%] [G loss: 11.268739] time: 0:04:34.196901
[Epoch 0/30] [Batch 118/816] [D loss: 2.541023, acc:  53%, op_acc: 85.00%] [G loss: 10.024487] time: 0:04:36.272342
[Epoch 0/30] [Batch 119/816] [D loss: 4.239997, acc:  53%, op_acc: 85.00%] [G loss: 10.190257] time: 0:04:38.428789
[Epoch 0/30] [Batch 120/816] [D loss: 1.250319, acc:  45%, op_acc: 90.00%] [G loss: 8.873822] time: 0:04:40.677705
[Epoch 0/30] [Batch 121/816] [D loss: 0.821938, acc:  48%, op_acc: 90.00%] [G loss: 10.544367] time: 0:04:42.825397
[Epoch 0/30] [Batch 122/816] [D loss: 3.707140, acc:  47%, op_acc: 80.00%] [G loss: 11.991008] time: 0:04:45.081185
[Epoch 0/30] [Batch 123/816] [D loss: 1.066811, acc:  52%, op_acc: 90.00%] [G loss: 9.383269] time: 0:04:47.277067
[Epoch 0/30] [Batch 124/816] [D loss: 2.341494, acc:  51%, op_acc: 95.00%] [G loss: 9.003647] time: 0:04:49.519693
[Epoch 0/30] [Batch 125/816] [D loss: 0.767260, acc:  50%, op_acc: 80.00%] 

[Epoch 0/30] [Batch 188/816] [D loss: 1.229764, acc:  47%, op_acc: 85.00%] [G loss: 10.551724] time: 0:07:11.243905
[Epoch 0/30] [Batch 189/816] [D loss: 1.067730, acc:  38%, op_acc: 95.00%] [G loss: 11.767858] time: 0:07:13.460252
[Epoch 0/30] [Batch 190/816] [D loss: 0.908456, acc:  55%, op_acc: 90.00%] [G loss: 11.473761] time: 0:07:15.676164
[Epoch 0/30] [Batch 191/816] [D loss: 0.305028, acc:  52%, op_acc: 100.00%] [G loss: 12.177772] time: 0:07:17.827966
[Epoch 0/30] [Batch 192/816] [D loss: 0.316976, acc:  55%, op_acc: 100.00%] [G loss: 11.020803] time: 0:07:20.077530
[Epoch 0/30] [Batch 193/816] [D loss: 0.737772, acc:  64%, op_acc: 90.00%] [G loss: 9.272228] time: 0:07:22.228046
[Epoch 0/30] [Batch 194/816] [D loss: 0.502091, acc:  50%, op_acc: 95.00%] [G loss: 10.103774] time: 0:07:24.461305
[Epoch 0/30] [Batch 195/816] [D loss: 0.340466, acc:  44%, op_acc: 100.00%] [G loss: 10.629846] time: 0:07:26.649367
[Epoch 0/30] [Batch 196/816] [D loss: 0.657730, acc:  53%, op_acc: 95.

...

[Epoch 1/30] [Batch 651/816] [D loss: 0.260708, acc:  64%, op_acc: 100.00%] [G loss: 11.074028] time: 0:51:35.202586
[Epoch 1/30] [Batch 652/816] [D loss: 0.588516, acc:  82%, op_acc: 95.00%] [G loss: 10.396732] time: 0:51:37.148202
[Epoch 1/30] [Batch 653/816] [D loss: 0.136972, acc:  88%, op_acc: 100.00%] [G loss: 11.017860] time: 0:51:39.082115
[Epoch 1/30] [Batch 654/816] [D loss: 1.344929, acc:  81%, op_acc: 95.00%] [G loss: 8.018446] time: 0:51:41.017450
[Epoch 1/30] [Batch 655/816] [D loss: 3.498665, acc:  93%, op_acc: 95.00%] [G loss: 8.534565] time: 0:51:42.954028
[Epoch 1/30] [Batch 656/816] [D loss: 1.218872, acc:  61%, op_acc: 95.00%] [G loss: 6.198162] time: 0:51:44.879548
[Epoch 1/30] [Batch 657/816] [D loss: 0.484843, acc:  75%, op_acc: 95.00%] [G loss: 8.397116] time: 0:51:46.818947
[Epoch 1/30] [Batch 658/816] [D loss: 0.254825, acc:  59%, op_acc: 100.00%] [G loss: 11.242249] time: 0:51:48.765619
[Epoch 1/30] [Batch 659/816] [D loss: 1.749221, acc:  81%, op_acc: 90.00%

[Epoch 1/30] [Batch 722/816] [D loss: 1.276715, acc:  69%, op_acc: 95.00%] [G loss: 9.156111] time: 0:53:52.435967
[Epoch 1/30] [Batch 723/816] [D loss: 2.043846, acc:  60%, op_acc: 90.00%] [G loss: 7.616390] time: 0:53:54.346589
[Epoch 1/30] [Batch 724/816] [D loss: 0.206489, acc:  75%, op_acc: 100.00%] [G loss: 8.276230] time: 0:53:56.295163
[Epoch 1/30] [Batch 725/816] [D loss: 0.260778, acc:  68%, op_acc: 100.00%] [G loss: 8.126133] time: 0:53:58.221417
[Epoch 1/30] [Batch 726/816] [D loss: 2.121241, acc:  89%, op_acc: 95.00%] [G loss: 6.905454] time: 0:54:00.152620
[Epoch 1/30] [Batch 727/816] [D loss: 0.180525, acc:  80%, op_acc: 100.00%] [G loss: 7.722677] time: 0:54:02.095532
[Epoch 1/30] [Batch 728/816] [D loss: 0.134135, acc:  89%, op_acc: 100.00%] [G loss: 9.164671] time: 0:54:04.019772
[Epoch 1/30] [Batch 729/816] [D loss: 0.240513, acc:  81%, op_acc: 100.00%] [G loss: 7.288038] time: 0:54:05.943512
[Epoch 1/30] [Batch 730/816] [D loss: 0.238915, acc:  92%, op_acc: 95.00%] 

[Epoch 1/30] [Batch 793/816] [D loss: 0.587795, acc:  93%, op_acc: 95.00%] [G loss: 8.169079] time: 0:56:09.757442
[Epoch 1/30] [Batch 794/816] [D loss: 0.155640, acc:  83%, op_acc: 100.00%] [G loss: 7.362740] time: 0:56:11.672354
[Epoch 1/30] [Batch 795/816] [D loss: 0.078424, acc:  95%, op_acc: 100.00%] [G loss: 7.642003] time: 0:56:13.613768
[Epoch 1/30] [Batch 796/816] [D loss: 0.076854, acc:  96%, op_acc: 100.00%] [G loss: 7.886372] time: 0:56:15.557747
[Epoch 1/30] [Batch 797/816] [D loss: 0.147763, acc:  87%, op_acc: 100.00%] [G loss: 8.371256] time: 0:56:17.500738
[Epoch 1/30] [Batch 798/816] [D loss: 0.080242, acc:  96%, op_acc: 100.00%] [G loss: 9.220791] time: 0:56:19.438111
[Epoch 1/30] [Batch 799/816] [D loss: 0.074091, acc:  95%, op_acc: 100.00%] [G loss: 8.142314] time: 0:56:21.381863
[Epoch 1/30] [Batch 800/816] [D loss: 0.088394, acc:  95%, op_acc: 100.00%] [G loss: 8.641461] time: 0:56:23.304668
SAVE
1_800-T1-T2.png
[Epoch 1/30] [Batch 801/816] [D loss: 0.042278, acc:

[Epoch 2/30] [Batch 48/816] [D loss: 0.555905, acc:  86%, op_acc: 95.00%] [G loss: 15.520541] time: 0:58:34.092447
[Epoch 2/30] [Batch 49/816] [D loss: 3.775545, acc:  70%, op_acc: 95.00%] [G loss: 14.260210] time: 0:58:36.033742
[Epoch 2/30] [Batch 50/816] [D loss: 0.129799, acc:  91%, op_acc: 100.00%] [G loss: 10.698509] time: 0:58:37.971514
[Epoch 2/30] [Batch 51/816] [D loss: 3.643901, acc:  84%, op_acc: 90.00%] [G loss: 10.327879] time: 0:58:39.910443
[Epoch 2/30] [Batch 52/816] [D loss: 0.090093, acc:  91%, op_acc: 100.00%] [G loss: 8.523052] time: 0:58:41.853990
[Epoch 2/30] [Batch 53/816] [D loss: 0.078660, acc:  93%, op_acc: 100.00%] [G loss: 11.866480] time: 0:58:43.795121
[Epoch 2/30] [Batch 54/816] [D loss: 0.298388, acc:  86%, op_acc: 95.00%] [G loss: 7.562788] time: 0:58:45.724927
[Epoch 2/30] [Batch 55/816] [D loss: 0.110884, acc:  88%, op_acc: 100.00%] [G loss: 9.591752] time: 0:58:47.659412
[Epoch 2/30] [Batch 56/816] [D loss: 0.888136, acc:  96%, op_acc: 85.00%] [G lo

[Epoch 2/30] [Batch 120/816] [D loss: 0.182667, acc:  74%, op_acc: 100.00%] [G loss: 8.673884] time: 1:00:53.478371
[Epoch 2/30] [Batch 121/816] [D loss: 0.139254, acc:  93%, op_acc: 100.00%] [G loss: 7.094369] time: 1:00:55.411307
[Epoch 2/30] [Batch 122/816] [D loss: 2.470126, acc:  88%, op_acc: 90.00%] [G loss: 7.012941] time: 1:00:57.339319
[Epoch 2/30] [Batch 123/816] [D loss: 0.089107, acc:  94%, op_acc: 100.00%] [G loss: 7.293682] time: 1:00:59.268150
[Epoch 2/30] [Batch 124/816] [D loss: 0.088325, acc:  93%, op_acc: 100.00%] [G loss: 8.741898] time: 1:01:01.202410
[Epoch 2/30] [Batch 125/816] [D loss: 1.508701, acc:  93%, op_acc: 95.00%] [G loss: 7.941904] time: 1:01:03.136716
[Epoch 2/30] [Batch 126/816] [D loss: 0.177599, acc:  79%, op_acc: 100.00%] [G loss: 6.467797] time: 1:01:05.059797
[Epoch 2/30] [Batch 127/816] [D loss: 0.136568, acc:  95%, op_acc: 95.00%] [G loss: 7.544991] time: 1:01:06.981039
[Epoch 2/30] [Batch 128/816] [D loss: 0.039571, acc:  98%, op_acc: 100.00%]